Описание задания: В рамках этого задания вам необходимо
разработать систему для управления библиотечным фондом, в
которой будут применены три паттерна проектирования: Фасад,
Заместитель и Легковес2
; Фасад – разработать единую точку входа для
взаимодействия с библиотечной системой, скрывая
сложность взаимодействия с подлежащими подсистемами
(например, учет книг, управление пользователями, выдача/
возврат книг)2
; Заместитель – реализовать паттерн заместителя для
контроля доступа к библиотечным данным, например, для
проверки наличия книги или прав на управление
пользовательскими записями без непосредственного
доступа к базе данных2
; Легковес – оптимизировать хранение информации о книгах,
используя паттерн Легковес, чтобы избежать дублирования
данных, например, для хранения общих характеристик книг
(например, названия, авторы), а не их копирования для
каждой записи о экземпляре книги.


Требования_
%2 Реализовать класс LibraryFacade, который будет включать
методы для работы с книгами, пользователями и системой
выдачи/возврата. Он должен скрывать сложности
реализации всех подлежащих подсистем2
2 Реализовать класс BookProxy (заместитель), который будет
проверять, доступна ли книга для выдачи или возврата, а
также обеспечивать защиту доступа к информации о книге2
W2 Реализовать класс BookFlyweight (легковес), который будет
хранить общую информацию о книгах (например, название,
автор, жанр) и использоваться для предотвращения
дублирования этих данных2
2 Система должна поддерживать несколько типов
пользователей (например, библиотекари и читатели), с
разными правами доступа2
q2 Описание каждого паттерна, его применение в контексте
задачи и объяснение архитектуры.

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List, Optional
from enum import Enum

# Легковес (Flyweight)
class BookFlyweight:
    """Класс для хранения общей информации о книгах (паттерн Легковес)"""
    def __init__(self, title: str, author: str, genre: str, year: int):
        self.title = title
        self.author = author
        self.genre = genre
        self.year = year
    
    def __str__(self):
        return f"{self.title} ({self.author}, {self.year}, {self.genre})"

class FlyweightFactory:
    """Фабрика легковесов для управления общими данными книг"""
    _flyweights: Dict[str, BookFlyweight] = {}

    @classmethod
    def get_flyweight(cls, title: str, author: str, genre: str, year: int) -> BookFlyweight:
        key = f"{title}_{author}_{genre}_{year}"
        if key not in cls._flyweights:
            cls._flyweights[key] = BookFlyweight(title, author, genre, year)
        return cls._flyweights[key]

    @classmethod
    def count_flyweights(cls) -> int:
        return len(cls._flyweights)

#  Заместитель (Proxy) 
class BookProxy:
    """Заместитель для контроля доступа к информации о книге"""
    def __init__(self, book_id: str, flyweight: BookFlyweight, is_available: bool = True):
        self.book_id = book_id
        self._flyweight = flyweight
        self._is_available = is_available
        self._real_book = None  # Здесь могла бы быть реальная книга в БД
    
    @property
    def title(self) -> str:
        return self._flyweight.title
    
    @property
    def author(self) -> str:
        return self._flyweight.author
    
    @property
    def is_available(self) -> bool:
        return self._is_available
    
    def check_availability(self) -> bool:
        """Проверка доступности книги"""
        return self._is_available
    
    def get_info(self, user: 'User') -> str:
        """Получение информации о книге с проверкой прав доступа"""
        if not user.has_access:
            return "Доступ к информации ограничен"
        return f"Книга ID: {self.book_id}, {self._flyweight}, {'Доступна' if self._is_available else 'На руках'}"
    
    def set_availability(self, available: bool, user: 'User') -> bool:
        """Изменение статуса книги с проверкой прав"""
        if not user.is_librarian:
            print("Ошибка: Недостаточно прав для изменения статуса книги")
            return False
        self._is_available = available
        return True

# Подсистемы библиотеки 
class UserRole(Enum):
    READER = "Читатель"
    LIBRARIAN = "Библиотекарь"

class User:
    """Класс пользователя библиотеки"""
    def __init__(self, user_id: str, name: str, role: UserRole):
        self.user_id = user_id
        self.name = name
        self.role = role
    
    @property
    def is_librarian(self) -> bool:
        return self.role == UserRole.LIBRARIAN
    
    @property
    def has_access(self) -> bool:
        return True  # В реальной системе могут быть сложные правила

class BookCatalog:
    """Подсистема каталога книг"""
    def __init__(self):
        self._books: Dict[str, BookProxy] = {}
    
    def add_book(self, book_id: str, flyweight: BookFlyweight) -> BookProxy:
        book = BookProxy(book_id, flyweight)
        self._books[book_id] = book
        return book
    
    def find_book(self, book_id: str) -> Optional[BookProxy]:
        return self._books.get(book_id)
    
    def list_books(self) -> List[BookProxy]:
        return list(self._books.values())

class UserManager:
    """Подсистема управления пользователями"""
    def __init__(self):
        self._users: Dict[str, User] = {}
    
    def add_user(self, user_id: str, name: str, role: UserRole) -> User:
        user = User(user_id, name, role)
        self._users[user_id] = user
        return user
    
    def find_user(self, user_id: str) -> Optional[User]:
        return self._users.get(user_id)

class LoanSystem:
    """Подсистема выдачи и возврата книг"""
    def __init__(self, catalog: BookCatalog):
        self._catalog = catalog
        self._loans: Dict[str, str] = {}  # book_id -> user_id
    
    def borrow_book(self, book_id: str, user_id: str) -> bool:
        book = self._catalog.find_book(book_id)
        if not book or not book.check_availability():
            return False
        book.set_availability(False, User(user_id, "", UserRole.LIBRARIAN))  # Упрощенный вызов
        self._loans[book_id] = user_id
        return True
    
    def return_book(self, book_id: str) -> bool:
        if book_id not in self._loans:
            return False
        book = self._catalog.find_book(book_id)
        if book:
            book.set_availability(True, User("sys", "System", UserRole.LIBRARIAN))
        del self._loans[book_id]
        return True

# Фасад (Facade) 
class LibraryFacade:
    """Фасад для упрощенного взаимодействия с библиотечной системой"""
    def __init__(self):
        self._catalog = BookCatalog()
        self._user_manager = UserManager()
        self._loan_system = LoanSystem(self._catalog)
    
    def add_book(self, book_id: str, title: str, author: str, genre: str, year: int) -> bool:
        flyweight = FlyweightFactory.get_flyweight(title, author, genre, year)
        self._catalog.add_book(book_id, flyweight)
        return True
    
    def register_user(self, user_id: str, name: str, is_librarian: bool = False) -> bool:
        role = UserRole.LIBRARIAN if is_librarian else UserRole.READER
        self._user_manager.add_user(user_id, name, role)
        return True
    
    def borrow_book(self, book_id: str, user_id: str) -> bool:
        return self._loan_system.borrow_book(book_id, user_id)
    
    def return_book(self, book_id: str) -> bool:
        return self._loan_system.return_book(book_id)
    
    def get_book_info(self, book_id: str, user_id: str) -> Optional[str]:
        book = self._catalog.find_book(book_id)
        user = self._user_manager.find_user(user_id)
        if not book or not user:
            return None
        return book.get_info(user)
    
    def list_available_books(self) -> List[str]:
        return [book.get_info(User("guest", "Гость", UserRole.READER)) 
                for book in self._catalog.list_books() 
                if book.is_available]

# Пользовательский код
def main():
    # Инициализация библиотечной системы
    library = LibraryFacade()
    
    # Регистрация пользователей
    library.register_user("user1", "Иванов Иван")
    library.register_user("lib1", "Петрова Мария", is_librarian=True)
    
    # Добавление книг
    library.add_book("001", "Война и мир", "Лев Толстой", "Роман", 1869)
    library.add_book("002", "Преступление и наказание", "Фёдор Достоевский", "Роман", 1866)
    library.add_book("003", "Война и мир", "Лев Толстой", "Роман", 1869)  # Другой экземпляр
    
    # Вывод информации о книгах
    print("\nДоступные книги:")
    for book_info in library.list_available_books():
        print(f" - {book_info}")
    
    # Выдача книги
    print("\nПопытка выдачи книги:")
    if library.borrow_book("001", "user1"):
        print(" - Книга 001 успешно выдана пользователю user1")
    else:
        print(" - Ошибка выдачи книги")
    
    # Проверка доступности
    print("\nСтатус книги 001:")
    print(f" - {library.get_book_info('001', 'user1')}")
    
    # Попытка выдать уже выданную книгу
    print("\nПопытка повторной выдачи книги 001:")
    if not library.borrow_book("001", "user1"):
        print(" - Книга уже на руках, выдача невозможна")
    
    # Возврат книги
    print("\nВозврат книги 001:")
    if library.return_book("001"):
        print(" - Книга успешно возвращена")
    
    # Проверка использования легковесов
    print(f"\nВсего уникальных книг (легковесов): {FlyweightFactory.count_flyweights()}")

if __name__ == "__main__":
    main()


Доступные книги:
 - Книга ID: 001, Война и мир (Лев Толстой, 1869, Роман), Доступна
 - Книга ID: 002, Преступление и наказание (Фёдор Достоевский, 1866, Роман), Доступна
 - Книга ID: 003, Война и мир (Лев Толстой, 1869, Роман), Доступна

Попытка выдачи книги:
 - Книга 001 успешно выдана пользователю user1

Статус книги 001:
 - Книга ID: 001, Война и мир (Лев Толстой, 1869, Роман), На руках

Попытка повторной выдачи книги 001:
 - Книга уже на руках, выдача невозможна

Возврат книги 001:
 - Книга успешно возвращена

Всего уникальных книг (легковесов): 2
